In [16]:
import _nb_utils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from config import CONFIG
from prompter import *
from data import *
from utils import *
from model import GenParams, GoogleAIStudioMLLM, OllamaMLLM

In [18]:
model_name = "gemma3:4b-it-qat"

llm = OllamaMLLM(model_name)

In [19]:
BY_MODEL = "LRASPP_MobileNet_V3"

In [20]:
gen_params = GenParams(
    seed=CONFIG["seed"],
    temperature=1.0
)

In [21]:
prompt_blueprint={
        "role": "0_baseline",
        "instruct": "0_baseline",
        "output_format": "0_baseline",
        "seeds_intro": "0_baseline",
        "seeds": "0_baseline",
        "query": "0_baseline",
}

In [22]:
score_level_range = [1, 5]

In [23]:
data_gen_prompt_builder = DataGenPromptBuilder(
    seed=CONFIG["seed"],
    prompt_blueprint=prompt_blueprint,
    by_model=BY_MODEL,
    seed_idxs=range(0, 10),
    score_level_range=[1, 5],
    num_seeds=5,
    num_outputs=4,
    rotate_prompts=False,
    jsonl_save_path=None
)

## Single Sample

In [24]:
seed_idxs = [0, 1, 2, 4, 5]
query_idxs = [0, 1, 2, 4, 5]

In [12]:
seeds = [str(get_one_answer_gt(by_model=BY_MODEL, idx=i, return_state=False)) for i in seed_idxs] 
data_gen_prompt_builder.build_data_gen_prompt(prompt_blueprint, seeds=seeds, num_outputs=5)

['You are an expert in evaluating the quality of textual descriptions of predictions produced against ground truth.\n',
 "Your task is to generate synthetic data <text, score> pairs, in which :\n- 'text' describes how well a prediction aligns with a ground truth.\n- 'score' (integer in the range [1, 5], the higher the better) indicating the quality of that prediction based on 'text'.\n",
 "Your output should be a JSON array of objects containing the 'text' and 'score' field.",
 'Here are a few examples of the desired output:\n',
 "{0: 'The ground truth AEROPLANE regions have been segmented in a coarser and incomplete way, especially regarding the wings, and two tiny AEROPLANE patches have been hallucinated on the right edge. The prediction mask for the ground truth PERSON region on the center-bottom-left is slightly more blob-like.\\n'}",
 '\n',
 "{1: 'The ground truth AEROPLANE regions have been captured with irregular and erratic boundaries, while some AEROPLANE area has been halluci

In [10]:
await data_gen_prompt_builder.generate_one_sample(
    model=llm,
    gen_params=gen_params,
    query_idxs=query_idxs,
    seed_idxs=seed_idxs
)

Wrong parsing to dict!
{'img_idx': [0, 1, 2, 4, 5], 'content': '{\n    "0": "The predicted bounding box for the ground truth BUILDING region is noticeably larger than the actual area, and the surrounding context is incorrectly labeled as ROAD. The edges of the predicted region appear somewhat diffuse, particularly at the top.\\n"\n  },\n  {\n    "1": "While the prediction mostly aligns with the ground truth VEHICLE region, there\'s a small but significant area of the predicted mask that extends beyond the true boundary, creating an \'over-prediction\'. The color segmentation is generally accurate, but some minor misclassifications of small elements as BACKGROUND exist.\\n"\n  },\n  {\n    "2": "The ground truth PLANT region\'s boundaries are sharply defined, but the prediction contains a substantial amount of false positives – specifically, regions of the background have been incorrectly labeled as PLANT. The prediction mask is significantly less detailed and has a higher level of nois